For a document with word counts x = (x₁, x₂, ..., xₙ) (where xᵢ is how many times word i appears), the probability of seeing this exact word count pattern if the document belongs to class Cₖ is:

​$$\log P(d|c) = \underbrace{\log \left( \frac{(\sum x_i)!}{\prod x_i!} \right)}_{\text{Constant } K} + \sum x_i \log P(x_i|c)$$
 
​
